# Sequence Classification using Recurrent Neural Networks(RNN)
In this homework, you will learn how to train a recurrent neural network for human action classification. RNN is designed handle sequential data. The network can incorporate both past history and current input. [This](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) is a very good tutorial. You should read it before you start.

## Setup
Please make sure you have h5py and torchnet installed
> pip install h5py

> pip install git+https://github.com/pytorch/tnt.git@master


In [1]:
import os
import numpy as np
import h5py

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as DD
import torchnet as tnt

use_cuda = torch.cuda.is_available()
print('use cuda: %s'%(use_cuda))
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor


use cuda: True


## Dataset
The data we are using is skeleton data, which indicates the 3D locations of body joints. In total, there are 25 body joints. It is collected by Kinect v2. To make it easier, each sequence have same number of frames. You need to classify 10 different actions. There are 2000 training sequences, 400 validation sequences, and 500 test sequences. Each sequence has 15 frames, each frame is a 75-dimension vector (3*25).

For your convenience, we provide the dataloader for you.


In [2]:
class Dataset(DD.Dataset):
    # subset can be: 'train', 'val', 'test'
    def __init__(self, data_path, subset='train'):
        super(Dataset, self).__init__()
        self.data_path = os.path.join(data_path, '%s_data.h5'%subset)
        self.subset = subset

        with h5py.File(self.data_path) as f:
            self.data = np.array(f['data'])

        if subset != 'test':
            self.label_path = os.path.join(data_path, '%s_label.h5'%subset)
            with h5py.File(self.label_path) as f:
                self.label = np.array(f['label'])

        self.num_sequences = self.data.shape[0]
        self.seq_len = self.data.shape[1]
        self.n_dim = self.data.shape[2]

    def __getitem__(self, index):
        seq = self.data[index]
        if self.subset != 'test':
            label = int(self.label[index])
            sample = {'seq': seq, 'label': label}
        else:
            sample = {'seq': seq}
        return sample

    def __len__(self):
        return self.num_sequences

trSet = Dataset('./data', subset='train')
valSet = Dataset('./data', subset='val')
tstSet = Dataset('./data', subset='test')

batch_size = 50
trLD = DD.DataLoader(trSet, batch_size=batch_size,
       sampler=DD.sampler.RandomSampler(trSet),
       num_workers=2, pin_memory=False)
valLD = DD.DataLoader(valSet, batch_size=batch_size,
       sampler=DD.sampler.SequentialSampler(valSet),
       num_workers=1, pin_memory=False)
tstLD = DD.DataLoader(tstSet, batch_size=batch_size,
       sampler=DD.sampler.SequentialSampler(tstSet),
       num_workers=1, pin_memory=False)

input_dim = trSet.n_dim
num_class = 10

## Model
Pytorch has implemented different types of recurrent layers for you. For this homework, you can use any type of RNNs as you want:
> torch.nn.RNN()

> torch.nn.LSTM()

> torch.nn.GRU()

You can check details for different types of recurrent layers here: [RNN](http://pytorch.org/docs/master/nn.html#torch.nn.RNN), [LSTM]( http://pytorch.org/docs/master/nn.html#torch.nn.LSTM), [GRU](http://pytorch.org/docs/master/nn.html#torch.nn.GRU)


### Implement a specific model
In this section, you need to implement a model for sequence classification. The model has following layers:
* A linear layer that can map features of 75-dimension to 100-dimension.
* 1 Layer LSTM layer with hidden size of 100
* A linear layer that goes from 100 to num_class (10). 

An LSTM layer takes an input of size of (batch_size, seq_len, fea_dim) and outputs a variable of shape (batch_size, seq_len, hidden_size). In this homework, the classification score for a sequence is the classification score for the last step of rnn_outputs.



In [3]:
# sequence classification model
class SequenceClassify(nn.Module):
    def __init__(self):
        super(SequenceClassify, self).__init__()
        
        ############## 1st To Do (10 points) ##############
        ###################################################
#         self.project_layer = None
#         self.recurrent_layer = None
#         self.classify_layer = None
        ###################################################
        ###################################################
        self.project_layer = nn.Linear(75, 100)
        self.recurrent_layer = nn.LSTM(100, 100)
        self.classify_layer = nn.Linear(100, 10)
        ###################################################
    
    # the size of input is [batch_size, seq_len(15), input_dim(75)]
    # the size of logits is [batch_size, num_class]
    def forward(self, input, h_t_1=None, c_t_1=None):
        # the size of rnn_outputs is [batch_size, seq_len, rnn_size]
        rnn_outputs, (hn, cn) = self.recurrent_layer(self.project_layer(input))
        # classify the last step of rnn_outpus
        # the size of logits is [batch_size, num_class]
        logits = self.classify_layer(rnn_outputs[:,-1])
        return logits

if use_cuda:
    model = SequenceClassify().cuda()
else:
    model = SequenceClassify()

## Train the model
After you have the dataloader and model, you can start training the model. Define a SGD optimizer with learning rate of 1e-3, and a cross-entropy loss function:

In [4]:
################ 2nd To Do  (5 points)##################
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)
criterion = nn.CrossEntropyLoss()

In [154]:
# run the model for one epoch
# can be used for both training or validation model
def run_epoch(data_loader, model, criterion, epoch, is_training, optimizer=None):
    if is_training:
        model.train()
        logger_prefix = 'train'
    else:
        model.eval()
        logger_prefix = 'val'

    confusion_matrix = tnt.meter.ConfusionMeter(num_class)
    acc = tnt.meter.ClassErrorMeter(accuracy=True)
    meter_loss = tnt.meter.AverageValueMeter()

    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        label = sample['label']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        input_label_var = Variable(label).type(LongTensor)

        # compute output
        # output_logits: [batch_size, num_class]
        output_logits = model(input_sequence_var)
        loss = criterion(output_logits, input_label_var)

        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        meter_loss.add(loss.data[0])
        acc.add(output_logits.data, input_label_var.data)
        confusion_matrix.add(output_logits.data, input_label_var.data)

    if (not is_training) or e%10 == 0:
        print('%s Epoch: %d  , Loss: %.4f,  Accuracy: %.2f'%(logger_prefix, epoch, meter_loss.value()[0], acc.value()[0]))
    return acc.value()[0]

num_epochs = 1
evaluate_every_epoch = 5
for e in range(num_epochs):
    run_epoch(trLD, model, criterion, e, True, optimizer)
    if e % evaluate_every_epoch == 0:
        run_epoch(valLD, model, criterion, e, False, None)   


/home/sbhal/.local/lib/python2.7/site-packages/ipykernel_launcher.py:31: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


train Epoch: 0  , Loss: 2.3088,  Accuracy: 10.25
val Epoch: 0  , Loss: 2.3080,  Accuracy: 9.00


## Kaggle Model Training

In [213]:
# sequence classification model
class SequenceClassify(nn.Module):
    def __init__(self):
        super(SequenceClassify, self).__init__()
        self.project_layer = nn.Linear(75, 200)
        self.recurrent_layer1 = nn.LSTM(200,400,num_layers=4,bidirectional=True,batch_first=True,dropout=0.4)
        self.rnn1 = nn.Sequential(
            nn.BatchNorm1d(800),
            nn.LeakyReLU(),
            nn.Linear(800, 10)
        )
        ###################################################
    
    # the size of input is [batch_size, seq_len(15), input_dim(75)]
    # the size of logits is [batch_size, num_class]
    def forward(self, input, h_t_1=None, c_t_1=None):
        # the size of rnn_outputs is [batch_size, seq_len, rnn_size]
        rnn_outputs, (hn, cn) = self.recurrent_layer1(self.project_layer(input))
    
        logits = self.rnn1(rnn_outputs[:,-1])
        return logits

if use_cuda:
    best_model = SequenceClassify().cuda()
    criterion = nn.CrossEntropyLoss().cuda()
else:
    best_model = SequenceClassify()
    criterion = nn.CrossEntropyLoss()
   
    
optimizer = torch.optim.Adam(best_model.parameters(), amsgrad=True)

In [214]:
import copy 
num_epochs = 101
evaluate_every_epoch = 5
# k_model = None
# best_accuracy = 0.0

for e in xrange(num_epochs):
    run_epoch(trLD, best_model, criterion, e, True, optimizer)
    if e % evaluate_every_epoch == 0:
        acc = run_epoch(valLD, best_model, criterion, e, False, None)
        if acc > best_accuracy:
            k_model = copy.deepcopy(best_model)
            best_accuracy = acc

run_epoch(valLD, k_model, criterion, e, False, None) 

/home/sbhal/.local/lib/python2.7/site-packages/ipykernel_launcher.py:31: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


train Epoch: 0  , Loss: 2.2294,  Accuracy: 13.70
val Epoch: 0  , Loss: 1.8826,  Accuracy: 20.75
val Epoch: 5  , Loss: 1.2830,  Accuracy: 57.25
train Epoch: 10  , Loss: 0.6470,  Accuracy: 77.85
val Epoch: 10  , Loss: 0.9162,  Accuracy: 71.25
val Epoch: 15  , Loss: 0.6433,  Accuracy: 81.00
train Epoch: 20  , Loss: 0.3630,  Accuracy: 87.65
val Epoch: 20  , Loss: 0.7109,  Accuracy: 78.50
val Epoch: 25  , Loss: 0.6757,  Accuracy: 79.75
train Epoch: 30  , Loss: 0.2494,  Accuracy: 90.85
val Epoch: 30  , Loss: 0.6966,  Accuracy: 81.25
val Epoch: 35  , Loss: 0.8253,  Accuracy: 79.50
train Epoch: 40  , Loss: 0.1537,  Accuracy: 94.50
val Epoch: 40  , Loss: 0.7145,  Accuracy: 82.00
val Epoch: 45  , Loss: 0.8289,  Accuracy: 80.50
train Epoch: 50  , Loss: 0.0762,  Accuracy: 97.35
val Epoch: 50  , Loss: 0.8044,  Accuracy: 82.25
val Epoch: 55  , Loss: 0.7912,  Accuracy: 82.50
train Epoch: 60  , Loss: 0.0999,  Accuracy: 96.50
val Epoch: 60  , Loss: 0.8427,  Accuracy: 82.50
val Epoch: 65  , Loss: 1.0084

/home/sbhal/.local/lib/python2.7/site-packages/ipykernel_launcher.py:48: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


86.0

In [215]:
best_accuracy = run_epoch(valLD, k_model, criterion, e, False, None) 

val Epoch: 100  , Loss: 0.7142,  Accuracy: 86.00


/home/sbhal/.local/lib/python2.7/site-packages/ipykernel_launcher.py:48: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/home/sbhal/.local/lib/python2.7/site-packages/ipykernel_launcher.py:31: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


## Submit your results on Kaggle

### Train a better model for action recognition!
Now it's your job to experiment with architectures, hyperparameters, loss functions, and optimizers to train a model that achieves better accuracy on the action recognition validation set.


### Testing the model and submit on Kaggle
Testing the model on the testing set and save the results as a .csv file. 
Please submitted the results.csv file generated by predict_on_test() to Kaggle(https://www.kaggle.com/t/934b80879bd741e6ac1967195604d4d9) to see how well your network performs on the test set. 
################ 3rd To Do  (20 points, the highest 3 entries get extra 10 points) ###############


In [216]:
# Use your best model to generate results on test set.

# generate csv file for test set
def predict_on_test(model, data_loader):
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    results=open('results.csv','w')
    count=0
    results.write('Id'+','+'Class'+'\n')
    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        scores = model(input_sequence_var)
        _, preds = scores.data.cpu().max(1)
        for i in range(len(preds)):
            results.write(str(count)+','+str(preds[i].numpy())+'\n')
            count+=1
    results.close()
    return count

count=predict_on_test(k_model, tstLD)
print(count)

1000


/home/sbhal/.local/lib/python2.7/site-packages/ipykernel_launcher.py:48: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


## Report the performance
################ 4th To Do  (5 points)##################

In this cell, you should write an explanation of what you did (network architecture, optimiziter, learning rate, epochs) and any visualizations or graphs that you make in the process of training and evaluating your network.
## Model:
I Tried various optimizer such as SGD,Adam,SparseAdam etc.. I Got the best accuracy with the Adam optimizer. I tried tuning it with weight_decay,lr and amsgrad. I got the best result with lr=1e-3 and amsgrad=True.
Model Architecture:

1.Linear Layer(75, 200)

2.LSTM(200,400,bidirectional) 

3.BatchNorm2d(800)

4.ReLU()

5.Linear(800)



I added Linear layer initially to get some simple features. After which I added the LSTM layer to learn features based on past input as well. It worked better but the model was still giving low accuracy. So, i added bidirectional to the layer and tried increasing the number of layers and hidden size of the LSTM layer. Increasing the hidden size increased accuracy but it took too much time to train. After adding bidirectional LSTM, the model worked very well on the data.
I tried changing the lr to 0.01, 0.1,0.0001 and 0.005. I got the best result with the lr=1e-3.


I submitted the model with the best validation accuracy of 86.0% to the kaggle competition. I got 81.4% accuracy on the dataset.

Report your Kaggle Performance here:
## Position : 23 Rohit Bhal 
## Accuracy: 81.4%